In [1]:
if 'sc' not in locals():
    from pyspark.context import SparkContext
    from pyspark.sql.context import SQLContext
    from pyspark.sql.session import SparkSession
    
    sc = SparkContext()
    sqlContext = SQLContext(sc)
    spark = SparkSession(sc)

In [2]:
%run 'Load & Clean Liar'.ipynb

File Name                                             Modified             Size
README                                         2017-04-23 03:33:54         1670
test.tsv                                       2017-04-23 03:21:28       301118
train.tsv                                      2017-04-23 03:21:12      2408165
valid.tsv                                      2017-04-23 03:21:22       301556



Liar train:
Index(['id', 'label', 'statement', 'subject', 'speaker', 'job', 'state',
       'party', 'barely_true_c', 'false_c', 'half_true_c', 'mostly_true_c',
       'pants_on_fire_c', 'venue'],
      dtype='object')

Liar test:
Index(['id', 'label', 'statement', 'subject', 'speaker', 'job', 'state',
       'party', 'barely_true_c', 'false_c', 'half_true_c', 'mostly_true_c',
       'pants_on_fire_c', 'venue'],
      dtype='object')

Liar train:
(10240, 14)

Liar test:
(1267, 14)

Liar train:
id                 0
label              0
statement          0
subject            0
speaker       

In [3]:
liar_train_df['score'] = (liar_train_df.mostly_true_c * 0.9 + liar_train_df.half_true_c * 0.5 + \
                        liar_train_df.barely_true_c * 0.2 + liar_train_df.false_c * -1 + \
                        liar_train_df.pants_on_fire_c * -2).astype('int')

liar_test_df['score'] = (liar_test_df.mostly_true_c * 0.9 + liar_test_df.half_true_c * 0.5 + \
                        liar_test_df.barely_true_c * 0.2 + liar_test_df.false_c * -1 + \
                        liar_test_df.pants_on_fire_c * -2).astype('int')

In [4]:
liar_train_df = liar_train_df.drop('mostly_true_c', axis=1)
liar_train_df = liar_train_df.drop('half_true_c', axis=1)
liar_train_df = liar_train_df.drop('barely_true_c', axis=1)
liar_train_df = liar_train_df.drop('false_c', axis=1)
liar_train_df = liar_train_df.drop('pants_on_fire_c', axis=1)

In [5]:
liar_test_df = liar_test_df.drop('mostly_true_c', axis=1)
liar_test_df = liar_test_df.drop('half_true_c', axis=1)
liar_test_df = liar_test_df.drop('barely_true_c', axis=1)
liar_test_df = liar_test_df.drop('false_c', axis=1)
liar_test_df = liar_test_df.drop('pants_on_fire_c', axis=1)

In [6]:
liar_train_df.shape, liar_test_df.shape

((8920, 10), (1111, 10))

In [7]:
liar_train_df['subject'] = liar_train_df.subject.str.split(',').apply(tuple)
liar_test_df['subject'] = liar_test_df.subject.str.split(',').apply(tuple)

In [8]:
liar_all_df = liar_train_df.append(liar_test_df)
liar_all_df.index = range(0, len(liar_train_df)+len(liar_test_df))
print(liar_all_df.shape)

(10031, 10)


In [9]:
liar_all_df.state = liar_all_df.state.str.lower
liar_all_df.state = liar_all_df.state.str.strip()

In [10]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
liar_all_subj_hot = liar_all_df.join(p.DataFrame(mlb.fit_transform(liar_all_df.subject),
                          columns=mlb.classes_,
                          index=liar_all_df.index))

liar_all_subj_hot

,id,label,statement,subject,speaker,job,state,party,venue,score,...,unions,urban,veterans,voting-record,water,wealth,weather,welfare,women,workers
0,2635.json,false,Says the Annies List political group supports ...,"(abortion,)",dwayne-bohac,State representative,NaN,republican,a mailer,-1,...,0,0,0,0,0,0,0,0,0,0
1,10540.json,half-true,When did the decline of coal start? It started...,"(energy, history, job-accomplishments)",scott-surovell,State delegate,NaN,democrat,a floor speech.,1,...,0,0,0,0,0,0,0,0,0,0
2,1123.json,false,Health care reform legislation is likely to ma...,"(health-care,)",blog-posting,,NaN,none,a news release,-99,...,0,0,0,0,0,0,0,0,0,0
3,9028.json,half-true,The economic turnaround started at the end of ...,"(economy, jobs)",charlie-crist,,NaN,democrat,an interview on CNN,17,...,0,0,0,0,0,0,0,0,0,0
4,12465.json,true,The Chicago Bears have had more starting quart...,"(education,)",robin-vos,Wisconsin Assembly speaker,NaN,republican,a an online opinion-piece,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10026,7334.json,half-true,Says his budget provides the highest state fun...,"(education,)",rick-scott,Governor,NaN,republican,a news conference,18,...,0,0,0,0,0,0,0,0,0,0
10027,9788.json,barely-true,Ive been here almost every day.,"(civil-rights, crime, criminal-justice)",jay-nixon,Governor,NaN,democrat,"on ABC's ""This Week""",1,...,0,0,0,0,0,0,0,0,0,0
10028,10710.json,barely-true,"In the early 1980s, Sen. Edward Kennedy secret...","(bipartisanship, congress, foreign-policy, his...",mackubin-thomas-owens,"senior fellow, Foreign Policy Research Institute",NaN,columnist,a commentary in The Providence Journal,0,...,0,0,0,0,0,0,0,0,0,0
10029,3186.json,barely-true,Says an EPA permit languished under Strickland...,"(environment, government-efficiency)",john-kasich,"Governor of Ohio as of Jan. 10, 2011",NaN,republican,a news conference,9,...,0,0,0,0,0,0,0,0,0,0


In [11]:
print("Label counts:", liar_all_subj_hot.label.nunique())
print("Job counts:", liar_all_subj_hot.job.nunique())
print("Party counts:", liar_all_subj_hot.party.nunique())
print("Venue counts:", liar_all_subj_hot.venue.nunique())

Label counts: 6
Job counts: 1273
Party counts: 24
Venue counts: 4119


In [12]:
liar_all_all_hot = liar_all_subj_hot.drop('speaker', axis=1)
liar_all_all_hot = liar_all_all_hot.drop('subject', axis=1)
liar_all_all_hot = liar_all_all_hot.drop('statement', axis=1)
liar_all_all_hot = liar_all_all_hot.drop('id', axis=1)
liar_all_all_hot = liar_all_all_hot.drop('job', axis=1)
liar_all_all_hot = liar_all_all_hot.drop('venue', axis=1)
liar_all_all_hot = liar_all_all_hot.drop('label', axis=1)
liar_all_all_hot = p.get_dummies(data=liar_all_all_hot, columns=['party', 'state'])
liar_all_all_hot.score = liar_all_all_hot.score.astype('int')

In [13]:
print(liar_all_all_hot.shape)
print(liar_all_all_hot.info)
print(type(liar_all_all_hot.score))


(10031, 168)
<bound method DataFrame.info of        score  10-news-tampa-bay  Alcohol  abc-news-week  abortion  \
0         -1                  0        0              0         1   
1          1                  0        0              0         0   
2        -99                  0        0              0         0   
3         17                  0        0              0         0   
4          0                  0        0              0         0   
...      ...                ...      ...            ...       ...   
10026     18                  0        0              0         0   
10027      1                  0        0              0         0   
10028      0                  0        0              0         0   
10029      9                  0        0              0         0   
10030      0                  0        0              0         0   

       afghanistan  after-the-fact  agriculture  animals  autism  ...  \
0                0               0            0      

In [16]:
liar_train_all_hot = liar_all_all_hot.iloc[0:8920, :]
liar_test_all_hot = liar_all_all_hot.iloc[8921:, :]
print(liar_all_all_hot.dtypes)

score                     int64
10-news-tampa-bay         int64
Alcohol                   int64
abc-news-week             int64
abortion                  int64
                          ...  
party_organization        uint8
party_republican          uint8
party_state-official      uint8
party_talk-show-host      uint8
party_tea-party-member    uint8
Length: 168, dtype: object


In [17]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=liar_train_all_hot.columns[1:].values,outputCol="features")
spDF_train = assembler.transform(sqlContext.createDataFrame(liar_train_all_hot[1:]))
spDF_test = assembler.transform(sqlContext.createDataFrame(liar_test_all_hot[1:]))

In [18]:
spDF_train = spDF_train.withColumn('score_float', spDF_train.score.cast('float')).drop("score").withColumnRenamed("score_float", "score")
spDF_test = spDF_test.withColumn('score_float', spDF_test.score.cast('float')).drop("score").withColumnRenamed("score_float", "score")
len(spDF_test.dtypes)

169

In [19]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol="features", labelCol="score")

lr_spark = lr.fit(spDF_train)

In [20]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.mllib.evaluation import RegressionMetrics, MulticlassMetrics

# Instantiate metrics object
metrics = RegressionMetrics(lr_spark.evaluate(spDF_test))

# Squared Error
print("MSE = %s" % metrics.meanSquaredError)
print("RMSE = %s" % metrics.rootMeanSquaredError)

# R-squared
print("R-squared = %s" % metrics.r2)

# Mean absolute error
print("MAE = %s" % metrics.meanAbsoluteError)

# Explained variance
print("Explained variance = %s" % metrics.explainedVariance)

AttributeError: 'LinearRegressionSummary' object has no attribute 'ctx'

In [ ]:
lr_evaluator = RegressionEvaluator(predictionCol="score", labelCol="features",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_spark.evaluate(spDF_test).predictions))
lr_evaluator = RegressionEvaluator(predictionCol="score", labelCol="features")
print("rmse on test data = %g" % lr_evaluator.evaluate(lr_spark.evaluate(spDF_test).predictions))

In [21]:
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder


dt = DecisionTreeRegressor(featuresCol="features", labelCol="score", varianceCol="variance")

paramGrid = ParamGridBuilder().addGrid(dt.maxDepth, [1,3,5,7]) \
                            .addGrid(dt.maxBins, [20, 50, 100, 200]) \
                            .addGrid(dt.minInstancesPerNode, [2, 5, 8, 15]).build()

crossval = CrossValidator(estimator=dt,
                          estimatorParamMaps=paramGrid,
                          evaluator=RegressionEvaluator(labelCol="score", predictionCol="prediction", metricName="r2"),
                          numFolds=10)

cvModel = crossval.fit(spDF_train)

In [27]:
cvModel.bestModel.featureImportances

SparseVector(167, {65: 0.0417, 99: 0.087, 104: 0.0113, 149: 0.06, 154: 0.3666, 160: 0.4334})

In [23]:
prediction = cvModel.transform(spDF_test)
RegressionEvaluator(labelCol="score", predictionCol="prediction", metricName="r2").evaluate(prediction)

0.11420507626767773

In [24]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder


rf = RandomForestRegressor(featuresCol="features", labelCol="score")

rf_paramGrid = ParamGridBuilder().addGrid(rf.maxDepth, [1,3,5,7]) \
                            .addGrid(rf.maxBins, [20, 50, 100, 200]) \
                            .addGrid(rf.numTrees, [2,5,10,25]) \
                            .addGrid(rf.minInstancesPerNode, [2, 5, 8, 15]).build()

rf_crossval = CrossValidator(estimator=rf,
                          estimatorParamMaps=rf_paramGrid,
                          evaluator=RegressionEvaluator(labelCol="score", predictionCol="prediction", metricName="r2"),
                          numFolds=10)

rf_cvModel = crossval.fit(spDF_train)

In [25]:
rf_prediction = rf_cvModel.transform(spDF_test)
RegressionEvaluator(labelCol="score", predictionCol="prediction", metricName="r2").evaluate(rf_prediction)

0.11420507626767773

In [26]:
rf_cvModel.bestModel.featureImportances

SparseVector(167, {65: 0.0417, 99: 0.087, 104: 0.0113, 149: 0.06, 154: 0.3666, 160: 0.4334})